In [1]:
import requests
import json
from tqdm import tqdm
from time import sleep

In [2]:
test_in = '/mnt/scratch/chenqu/aol/original/session_test.last_9.txt'
test_out = '/mnt/scratch/chenqu/aol/original/session_test.gen.encoding.txt'

In [3]:
url = 'https://api.msturing.org/gen/encode'
params = {'key': 'value'}
headers = {'Content-Type': 'application/json',
           'Ocp-Apim-Subscription-Key': '5d0ffce20b164037945a43206857e84a'}
# queries_for_request = {"queries": ["Microsoft", "Bing.com", "Azure Cloud Services"]}
# r = requests.post(url, headers=headers, json=queries_for_request)

In [4]:
# send a request every N sessions
N = 1
counter = 0
all_queries = []
query_num = []
session_ids = []
error_all_queries = []
error_session_ids = []
error_query_num = []
with open(test_in) as fin, open(test_out, 'a') as fout:
    for line in tqdm(fin):
        counter += 1
        session = json.loads(line.strip())
        session_id = session['session_id']
        session_ids.append(session_id)
        queries = [turn['text'] for turn in session['query']]
        query_num.append(len(queries))
        all_queries.extend(queries)
        
        if counter == N:
            queries_for_request = {'queries': all_queries}
            r = requests.post(url, headers=headers, json=queries_for_request)
            query_vectors = []
            if r.status_code == 200:
                res = r.json()
                for item in res:
                    if item['status'] == 'SUCCESS':
                        query_vectors.append(item['vector'])
                    else:
                        print('failed query')
                        print('session_ids', session_ids)
                        print('response', item)
                
                for num, session_id in zip(query_num, session_ids):
                    each_query_vectors = query_vectors[:num]
                    query_vectors = query_vectors[num:]
                    fout.write(json.dumps({'vectors': each_query_vectors, 'session_id': session_id}) + '\n') 

            else:
                print('wrong status code', r.status_code)
                print('response content', r.content)
                print('session_ids', session_ids)
                print('queries', queries)
                error_all_queries.extend(all_queries)
                error_session_ids.extend(session_ids)
                error_query_num.extend(query_num)
                
                
            all_queries, query_num, session_ids = [], [], []
            counter = 0
            
        sleep(0.1)

9it [00:02,  3.27it/s]


In [5]:
print('dealing with error queries')
with open(test_out, 'a') as fout:
    for num, session_id in tqdm(zip(error_query_num, error_session_ids)):
        error_queries = error_all_queries[:num]
        error_all_queries = error_all_queries[num:]
        queries_for_request = {'queries': error_queries}
        r = requests.post(url, headers=headers, json=queries_for_request)
        query_vectors = []
        if r.status_code == 200:
            res = r.json()
            for item in res:
                if item['status'] == 'SUCCESS':
                    query_vectors.append(item['vector'])
                else:
                    print('failed query')
                    print('session_id', session_id)
                    print('response', item)

            fout.write(json.dumps({'vectors': query_vectors, 'session_id': session_id}) + '\n')
            
        else:
            print('wrong status code', r.status_code)
            print('response content', r.content)
            print('session_id', session_id)
            print('error_queries', error_queries)
            
        sleep(0.1)


0it [00:00, ?it/s]

dealing with error queries



1it [00:00,  3.15it/s]
2it [00:00,  3.20it/s]
3it [00:00,  3.24it/s]
4it [00:01,  3.27it/s]
5it [00:01,  3.29it/s]
6it [00:01,  3.29it/s]
7it [00:02,  3.33it/s]
8it [00:02,  3.32it/s]
9it [00:02,  3.32it/s]
10it [00:03,  3.33it/s]
11it [00:03,  3.34it/s]
12it [00:03,  3.34it/s]
13it [00:03,  3.34it/s]
14it [00:04,  3.35it/s]
15it [00:04,  3.35it/s]
16it [00:04,  3.35it/s]
17it [00:05,  3.34it/s]
18it [00:05,  3.34it/s]
19it [00:05,  3.37it/s]
20it [00:05,  3.36it/s]
21it [00:06,  3.37it/s]
22it [00:06,  3.37it/s]
23it [00:06,  3.37it/s]
24it [00:07,  3.36it/s]
25it [00:07,  3.36it/s]
26it [00:07,  3.36it/s]
27it [00:08,  3.36it/s]
28it [00:08,  3.38it/s]
29it [00:08,  3.37it/s]
30it [00:08,  3.39it/s]
31it [00:09,  3.37it/s]
32it [00:09,  3.36it/s]
33it [00:09,  3.34it/s]
34it [00:10,  3.34it/s]
35it [00:10,  3.34it/s]
36it [00:10,  3.34it/s]
37it [00:11,  3.34it/s]
38it [00:11,  3.34it/s]
39it [00:11,  3.34it/s]
40it [00:11,  3.36it/s]
41it [00:12,  3.35it/s]
42it [00:12,  3.33it/s]
